In [3]:
#Written by Dr. Alex Duncan 
#Depends on the invariance of the resultant under “change of base” or ring homomorphisms of the coefficient ring.

import time
import datetime

# This is more direct, but probably slower.
def uniform_hypergraph_charpoly_naive( n, edges):
    d = len(edges[0])
    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)
    t_poly = f"t*({fermat}) - {d}*({edge_poly})"

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[t][{variable_names}]')
    F = macaulay2(t_poly)
    disc = F.discriminant()
    return disc.sage()

# This computes the polynomial by multiple evaluation
# followed by Lagrange interpolation
# (maybe faster for larger examples?)
# It gives an estimated time remaining counter.
def uniform_hypergraph_charpoly( n, edges, estimate_time = False):
    d = len(edges[0])

    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[{variable_names}]')

    total_degree = n*(d-1)^(n-1)
    points = []
    start_time = time.time()
    for t in range(total_degree+1):
        t_poly = f"{t}*({fermat}) -  {d}*({edge_poly})"
        F = macaulay2(t_poly)
        disc = F.discriminant()
        points.append([t,disc.sage()])
        if estimate_time: # totally optional
            time_passed = time.time() - start_time
            time_remaining = time_passed*(total_degree-t)/(t+1)
            print(f"Time Remaining: {datetime.timedelta(seconds=time_remaining)}")
    R.<t> = PolynomialRing(QQ)
    return R.lagrange_polynomial(points)

t=var("t")
n=4
edges = [[0,1,2],[0,1,3],[0,2,3],[1,2,3]]

poly = uniform_hypergraph_charpoly(n,edges)
f=expand(factor(poly))
f = f/f.leading_coefficient()
print(f)

t^32 - 24*t^29 - 42*t^28 + 168*t^26 + 552*t^25 + 171*t^24 - 1264*t^23 - 2688*t^22 - 1584*t^21 + 6564*t^20 + 10944*t^19 - 1872*t^18 - 19824*t^17 - 22617*t^16 + 16992*t^15 + 56672*t^14 - 4536*t^13 - 70938*t^12 - 4416*t^11 + 53736*t^10 + 4680*t^9 - 25011*t^8 - 1776*t^7 + 6624*t^6 + 256*t^5 - 768*t^4
